# Folium Heat Map

In [1]:
from lxml import objectify
import folium
from folium.plugins import HeatMap
import pandas as pd
import numpy as np

In [2]:
#tcx_file = 'input/5501640999.tcx'
tcx_file = 'input/5782299709.tcx'
namespace = 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'
tree = objectify.parse(tcx_file)
root = tree.getroot()
activity = root.Activities.Activity

In [3]:
def latitude_values():
    return [float(x.text) for x in root.xpath('//ns:Position/ns:LatitudeDegrees', namespaces={'ns': namespace})]

def longitude_values():
    return [float(x.text) for x in root.xpath('//ns:Position/ns:LongitudeDegrees', namespaces={'ns': namespace})]

In [4]:
coords_pedro = list(zip(latitude_values(), longitude_values()))

In [5]:
meanlat = np.mean([i[0] for i in coords_pedro])
meanlon = np.mean([i[1] for i in coords_pedro])

In [6]:
# Initialize map
mapa = folium.Map(location=[meanlat, meanlon],
                  tiles='Cartodb Positron', zoom_start=18)

# Add heat
mapa.add_children(HeatMap(data=coords_pedro, max_zoom=20, radius=6, blur=5, max_val=10))

# Display map
mapa

# Create dataframe & export to csv

In [7]:
def heart_rate_values():
    return [int(x.text) for x in root.xpath('//ns:HeartRateBpm/ns:Value', namespaces={'ns': namespace})]

def time_values():
    return [x.text for x in root.xpath('//ns:Time', namespaces={'ns': namespace})]

In [8]:
# Zip in a list and create a df
list_coords_pedro = list(zip(time_values(), latitude_values(), longitude_values(), heart_rate_values()))
df_coords_pedro = pd.DataFrame(list_coords_pedro, columns=['time', 'latitude', 'longitude', 'heart_rate'])

In [10]:
# Slice time with the format 'hh:mm:ss'
df_coords_pedro['time'] = df_coords_pedro.time.apply(lambda x: x[11:19])

In [11]:
df_coords_pedro.head(5)

,time,latitude,longitude,heart_rate
0,20:38:51,48.831273,2.272584,112
1,20:38:52,48.831268,2.272582,112
2,20:38:53,48.831276,2.272587,112
3,20:38:54,48.831290,2.272590,112
4,20:38:55,48.831304,2.272587,112


In [ ]:
# Export to csv
df_coords_pedro.to_csv('coords_pedro.csv', index=False)